# Making static maps with GeoPandas and Matplotlib

Notebook developed by Sam Maurer, with some of the content adapted from GeoPandas tutorials [[1](https://geopandas.org/mapping.html), [2](https://geopandas.org/gallery/plotting_with_geoplot.html)] and the [replication notebooks](https://github.com/ual/displacement-chapter-replication) for a book chapter.

----

As with non-geospatial visualizations, there is an ecoystem of Python tools for making maps  powered by **Matplotlib**. Generally, the approach is either to treat geospatial coordinates directly as x-y values, or to re-project them on the fly and plot the projected values.

These are the go-to tools for making static maps:

**GeoPandas** has commands for generating maps directly from a GeoDataFrame.
- https://geopandas.org/mapping.html (user's guide)
- https://geopandas.org/reference.html#geopandas.GeoDataFrame.plot (all the plotting arguments)
- https://geopandas.org/gallery/index.html (gallery of examples)

**GeoPlot** has templates for a variety of common map types, and lets you set map projections on the fly. It's aiming to be the "Seaborn" of map generation.
- https://residentmario.github.io/geoplot/ (user's guide)
- https://residentmario.github.io/geoplot/api_reference.html (all the templates + arguments)
- https://residentmario.github.io/geoplot/gallery/index.html (gallery of examples)

**CartoPy** provides behind-the scenes building blocks, for example for projecting map data or for turning it into shapes that can be displayed by Matplotlib. But it can be difficult to work with directly.
- https://scitools.org.uk/cartopy (user's guide)

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

## 1. Load data

We'll start off by looking at a data file of country outlines (and some other characteristics) that's provided with GeoPandas. It comes from [Natural Earth](https://www.naturalearthdata.com), which is a nice resource for map data.

In [ ]:
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
countries.head()

## 2. Making a map directly from GeoPandas

API reference for `geopandas.GeoDataFrame.plot()`: https://geopandas.org/reference.html#geopandas.GeoDataFrame.plot

In [ ]:
countries.plot()

plt.show()  # invoking Matplotlib/PyPlot, as we've always done

In [ ]:
# Cleaning up the formatting a bit:

countries.plot(color='none', edgecolor='black', linewidth=1, figsize=(14,10))

plt.show()

It looks a little distored. **What's the coordinate reference system?**

In [ ]:
countries.crs

https://epsg.io/4326 = WGS84 lat/lon

**GeoPandas does not re-project your data when making a map.** When you call `geopandas.GeoDataFrame.plot()`, it plots the coordinate values directly on an x-y axis.

Incidentally, treating latitude and longitude as x-y coorindates has a name: it's called a plate carée or equirectangular projection: https://en.wikipedia.org/wiki/Equirectangular_projection

We *could* change the projection by changing the GeoDataFrame's coordinate reference system, but we're not going to right now -- there's another tool that lets you apply projections on the fly.

## 3. Making a map using GeoPlot

GeoPlot has templates for a variety of common map types, and lets you set map projections on the fly. It's aiming to be the "Seaborn" of map generation.

It's not installed in DataHub, so we'll install it in the following cell. (You will have to do this each time your server restarts.) `%%capture` suppresses output from the cell, just to keep the notebook tidy.

In [ ]:
%%capture
!pip install geoplot;

In [ ]:
import geoplot

All the **GeoPlot templates** and their arguments: https://residentmario.github.io/geoplot/api_reference.html

In [ ]:
# A polyplot shows simple shapes. The default settings are pretty nice looking.

geoplot.polyplot(countries, figsize=(14,10))

plt.show()

### Setting the map projection

In GeoPlot, you can set the map projection on the fly! Here are all the built-in options: https://residentmario.github.io/geoplot/api_reference.html#projections

In [ ]:
ax = geoplot.polyplot(countries, projection=geoplot.crs.Robinson())

ax.outline_patch.set_visible(True)  # show the map outline

plt.show()

### Exercises

1. Adapt one of the GeoPlot maps to show just one continent (hint: try filtering the rows in the GeoDataFrame)
  

2. A **choropleth** map shades in areas according to the value of a related variable. Try doing this with your continent map, coloring in countries according to their population or GDP (these are columns in the data table). 

  You'll need to use the `geoplot.choropleth()` function instead of `polyplot()`, and add a parameter `hue='<colname>'` indicating which column to draw data from.
  

3. Using the whole world, try setting the GeoPlot projection to `Orthographic()`

## 4. Multiple data layers

Often, you'll want to show multiple datasets on a map at the same time. You can mix and match the libraries, since they're all working on top of Matplotlib. But a big challenge is that **all the coordinate reference systems and map projections have to match up** -- or at least be close enough that the data appears in the correct place. 

The best strategy is to stick to one of these two approaches:

1. **Use GeoPlot for all the layers**, and set them to the same projection.
  
  
2. Or, **don't set any projections**, and make sure all the layers have a similar coordinate reference system (e.g., lat-lon). 

In [ ]:
# Let's load the storefront data that we downloaded in `spatial-data.ipynb`

storefronts = gpd.read_file('all56_nACSxMSA__41860.0.geojson')

In [ ]:
# Plot a map

ax = geoplot.pointplot(storefronts, s = 1)  # size of each point

ax.set_title("Storefronts in the Bay Area")

plt.show()

### Adding coastline

The biggest single improvement we can make is probably to add in the coastline, which will give people familiar with the Bay Area a clearer sense of what they're looking at.

An aside about **locating background map data**:

When you start visualizing geosptial data, all of a sudden you have to find appropriate *background* datasets to include in your maps: country borders, states, counties, etc. 

[Natural Earth](https://www.naturalearthdata.com), mentioned above, is a nice resource for this. So is the U.S. Census [TIGER](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html) data. That stands for Topologically Integrated Geographic Encoding and Referencing, obviously.

Generally, you want to look for what's called *"cartographic"* shapefiles for this: not the administrative boundaries, which might extend out into the ocean, but the common-sense boundaries that people are most familiar with.

In [ ]:
# Download a cartographic shapefile of the 50 states, which will include coastline

import requests
url = 'https://www2.census.gov/geo/tiger/GENZ2016/shp/cb_2016_us_state_500k.zip'

with open('cb_2016_us_state_500k.zip', 'wb') as f:
    r = requests.get(url)
    f.write(r.content)

In [ ]:
outlines = gpd.read_file('zip://cb_2016_us_state_500k.zip')

In [ ]:
# Plot the coastline data we just loaded
ax = geoplot.polyplot(outlines, 
                      projection = geoplot.crs.AlbersEqualArea(),
                      extent = (-123, 36.9, -121.4, 38.7))  # lower left, upper right

# Overlay the storefront points
geoplot.pointplot(storefronts, 
                  projection = geoplot.crs.AlbersEqualArea(),
                  ax = ax,  # add it to the axes created above
                  s = 1)

ax.set_title("Storefronts in the Bay Area")

plt.show()

### Exercises

4. Load the earthquakes data we've used in earlier demos.

  Add them on top of the state outlines (instead of the storefronts), and change the map extent to show a larger area. Or, add them on top of the world map.
  
  
5. If you'd like more practice with this, try using a **spatial join** (see `spatial-data.ipynb`) to calculate how many earthquakes occurred within the boundaries of each state, or country of the world. Make a choropleth map coloring in the states or countries based on earthquake frequency.

  To include earthquakes off the coast, you could use GeoPandas to create a **buffer** around each country, and then perform the join using the buffered shapes.

## 5. Bonus: Multiple layers using Matplotlib

This code illustrates how you can mix-and-match map shapes with other kinds of data visualizations and annotations in Matplotlib. It can take a while to work out all the syntax to accomplish things like this, but it's quite powerful.

In [ ]:
# Pull latitude and longitude out into normal DataFrame columns

storefronts['lon'] = storefronts.geometry.x
storefronts['lat'] = storefronts.geometry.y

https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hexbin.html

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# Plot coastline using GeoPandas
outlines.plot(ax=ax, color='whitesmoke', edgecolor='black')

# Plot hexbins of storefronts using Matplotlib
hb = plt.hexbin(storefronts.lon, storefronts.lat, bins='log', mincnt=1, gridsize=180)

# Add a key
cb = fig.colorbar(hb, ax=ax)
cb.set_label('Count')

# Add some annotations
plt.annotate(s='San Francisco', xy=(-122.55, 37.8), bbox=dict(facecolor='white'))
plt.annotate(s='Berkeley', xy=(-122.3, 37.9), bbox=dict(facecolor='white'))
plt.annotate(s='Oakland', xy=(-122.25, 37.8), bbox=dict(facecolor='white'))

# Title and bounds
ax.set_title('Storefronts in the Bay Area')
ax.set_xlim((-122.6, -121.9))
ax.set_ylim((37.4, 38.2))

plt.show()